# **Data download and reorganisation**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import matplotlib
import os
from nltk.corpus import stopwords #Stopwords of course
train_df = pd.read_csv('/kaggle/input/traincsv/train.csv').fillna(' ')
train_df.sample(10, random_state=13)
#reorganisation of dataset: 6 columns --> 1 column
train_df = train_df.assign(sum =0)
train_df.loc[(train_df['toxic'] == 1) |(train_df['severe_toxic'] == 1 )|(train_df['obscene'] == 1) |
             (train_df['threat'] == 1) |(train_df['insult'] == 1 )|(train_df['identity_hate'] == 1) , 'sum'] = 1
train_df.drop(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis = 1, inplace = True)
train_df.info()
train_df.describe()
train_df.head(100)

stop_words = set(stopwords.words("english"))
stop_words.add("!!!")
print(stop_words)

# Notes -------------------------------------------------------------------------------------
# A really good Pandas function resource
# https://www.analyticsvidhya.com/blog/2021/05/pandas-functions-13-most-important/

# Just a tutorial on NN, doesn't seem to be useful
# https://www.youtube.com/watch?v=tMrbN67U9d4 left off at 3:17

In [ ]:
train_df.loc[train_df['sum']==1].sample(10, random_state=10)

In [ ]:
df2 = train_df[train_df['sum'] == 1 ][0:2000] #here you can set up the size of training data (affects the time of training)
df3 = train_df[train_df['sum'] == 0 ][0:2000] #here you can set up the size of training data (affects the time of training)
#this doesn't give the best accuracy and done for speed only. set up it to 20000 and 20000 to have accuracy =~ 90%
frames = [df2, df3]
result = pd.concat(frames)
result['comment_text'] = result['comment_text'].replace({'@':''}, regex=True)
result['comment_text'] = result['comment_text'].replace({'\n':''}, regex=True)
result
result['sum'].plot(kind='hist', title='test');
result['comment_text'] = result['comment_text'].str.lower()

In [ ]:
y = result['sum'].values

# **Tokenization and Stemming**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize #sent=sentence
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize #For tagging words

In [ ]:
from copy import deepcopy
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
  
ps = PorterStemmer()

comments = result['comment_text'].values

def to_tokens(text):
    org_comments = []
    for i in text:
        d = word_tokenize(i)
        org_comments.append(d)
        for w in d:
            if w in stop_words:
                d.remove(w)
            if ((bool(re.match('^(?=.*[a-zA-Z])', w)))==False): #delete punctuation with regular expressions
                d.remove(w)
            else:
                w = ps.stem(w)
    return org_comments

org_comments = to_tokens(comments)
org_comments2 = deepcopy(org_comments)

# **Token work and Pad Text Data**

In [ ]:
from collections import Counter
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
from tokenizers import Tokenizer
from copy import deepcopy

# Functions 
def process_content():  #Speech tagging function
    try:
        for i in unfiltered_sent1:
            tagged = nltk.pos_tag(unfiltered_sent1)
        print(tagged)
    except Exception as e:
        print(str(e))

def flatten(input):
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list

def idGenerator(array):
    set_array = set(array) #set() removes any repeating words
    #Sets can't be subscripted, therefor, we need to move it to a new subscriptable list
    word_list = list(set_array)
    id_list = [0]*len(word_list)
    for i in range(len(word_list)):
        id_list[i] = i; #i changes i+1 to i, otherwise training dimensions problems
    zip_iterator = zip(word_list, id_list)
    return dict(zip_iterator)
        
def str_to_int(org_comments): 
    AllWords = flatten(org_comments)
    dict1 = idGenerator(AllWords)
    #replacing strings with  integers
    for i in org_comments:
        a = 0
        for j in i:
            i[a] = dict1.get(j)
            a+=1
    return org_comments

org_comments = str_to_int(org_comments)

In [ ]:
#what is the longest comment?
arr_of_length = []
for i in org_comments:
    arr_of_length.append(len(i))
max_number = max(arr_of_length) #the max length of comments
x_train_val = sequence.pad_sequences(org_comments, maxlen=max_number+1) #x_train_values are ready

# **Embedding Matrix and Embedding Layer**

We can greatly improve embeddings by learning them using a neural network on a supervised task. The embeddings form the parameters — weights — of the network which are adjusted to minimize loss on the task. The resulting embedded vectors are representations of categories where similar categories — relative to the task — are closer to one another.

Distributed word representation approaches such as Word2vec usually learn word vectors from a large corpus based on the distributional hypothesis.

**Word2Vec try**

In [ ]:
from gensim.models import word2vec
# Set values for various parameters
#print(org_comments2)
num_features = 200    # Word vector dimensionality                      
min_word_count = 1   # Minimum word count, have to set up this =1 otherwise does not work                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)
modelWv = word2vec.Word2Vec(org_comments2, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)
#modelWv.init_sims(replace=True) # marks the end of training to speed up the use of the model
#the model is created and trained

In [ ]:
words = list(modelWv.wv.index_to_key)
#print(words)

In [ ]:
#print(modelWv.wv['Nope']) #some vector for some word in the model

In [ ]:
from keras.layers import Embedding
weights = modelWv.wv.vectors

embedding_layer = Embedding(
    input_dim=weights.shape[0], output_dim=weights.shape[1],
    weights=[weights], trainable=True #i changed it to True to experiment with the NN, but it must be False
        )

In [ ]:
#let's see what vectors look like
modelWv.wv.vectors

# **Word2Vec 2d illustration**

In [ ]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

# train model
model2 = Word2Vec(org_comments2, min_count=10, vector_size=2)

# fit a 2d PCA model to the vectors
X = model2.wv.vectors
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
fig = pyplot.figure(figsize=(10,10))
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model2.wv.key_to_index)
t = 0
for i, word in enumerate(words):
    if (t%1==0):
        pyplot.annotate(word, xy=(result[i,0], result[i,1]))
    t = t + 1
pyplot.show()

# **Combining layers into the Keras model**

In [ ]:
filters = 250
kernel_size = 3
hidden_dims = 250

In [ ]:
#keras model, adding its layers
keras_model = Sequential()
keras_model.add(embedding_layer)
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
keras_model.add(MaxPooling1D())
keras_model.add(Conv1D(filters,
                 5,
                 padding='valid',
                 activation='relu'))
keras_model.add(GlobalMaxPooling1D())
keras_model.add(Dense(hidden_dims, activation='relu'))
keras_model.add(Dropout(0.2))
keras_model.add(Dense(1, activation='sigmoid'))

In [ ]:
keras_model.summary()

In [ ]:
keras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# **Training the Keras model**

In [ ]:
#training
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y, test_size=0.15, random_state=1)
batch_size = 32
epochs = 3

keras_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val))

# **Model evaluation**

In [ ]:
keras_model.evaluate(x_val, y_val, batch_size=32)

# **Our custom tests; result visualisation**

In [ ]:
from nltk.stem import PorterStemmer
import re
ps = PorterStemmer()
tester2 = ["What do you think about this idea?", "Let's agree to disagree", "Just shut up, you opinion is stupid", "Eat a plate of shit", "Hello people have a good day", "Let's play the guitar", "Stop this crap this is crazy", "My dog is cleverer than you"]
tester2 = pd.DataFrame (tester2, columns = ['Comments'])
tester2['Comments'] = tester2['Comments'].replace({'@':''}, regex=True)
tester2['Comments'] = tester2['Comments'].replace({'\n':''}, regex=True)
tester2['Comments'] = tester2['Comments'].str.lower()
tester = tester2['Comments'].values
org_tester = to_tokens(tester)
org_tester3 = str_to_int(org_tester)
x_testing = sequence.pad_sequences(org_tester3, maxlen=max_number+1)
y_testing = keras_model.predict(x_testing, verbose = 1, batch_size=1)

In [ ]:
tester2['Toxic'] = ['not toxic' if x < 0.50 else 'toxic' for x in y_testing]
tester2[['Comments', 'Toxic']].head(20)

In [ ]:
#probabilities for all results
y_testing

# **How to save and then download a trained model**

In [ ]:
keras_model.save('/kaggle/working/')

In [ ]:
#after saving the model it is possible to load it here
from tensorflow import keras
keras_model = keras.models.load_model('/kaggle/working/')